<a href="https://colab.research.google.com/github/armandordorica/MIE1516_A1_Variable_Elimination/blob/master/A2_example_from_warmup_All_3_queries_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

### Global Functions

In [0]:
def remove_irrelevant_variable(df):
  """
  If a dataframe has a column of just 1s, i.e. evidence has been observed then 
  remove those columns from tabular representation
  """
  list(df.columns).pop(-1)
  vars = list(df.columns)
  vars.pop(-1)

  indices_to_pop = []

  for i in range(0, len(vars)):
    # print(i)
    # print(str(vars[i]))
    # print(df[str(vars[i])])
    if df[str(vars[i])].sum() == df[str(vars[i])].count(): #if there's a column of 1s 
      print("There's a column of 1s for variable {}".format(vars[i]))
      print("Removing {}".format(str(vars[i])))
      indices_to_pop.append(i)

  for j in range(0, len(vars)):
    print(j)
    print(str(vars[j]))
    print(df[str(vars[j])])
    if (df[vars[j]]==0).astype(int).sum() == df[str(vars[j])].count(): ## if there's a column of zeros 
      print("There's a column of 0s for variable {}".format(vars[j]))
      print("Removing {}".format(str(vars[j])))
      indices_to_pop.append(j)

  for i in range(len(indices_to_pop)):
    vars.pop(indices_to_pop[i])


  vars.append("Pr")
  df[vars]
  return df[vars]

def df_to_cpd(df):
  df = remove_irrelevant_variable(df)
  vars = list(df.columns)
  vars.pop(-1)
  vars = vars[0]
  variable_card = len(list(df.T.columns))
  values = list(df['Pr'])
  df_cpd = TabularCPD(variable=vars, variable_card=variable_card, values=[values])
  return df_cpd

def delete_df_from_list(factors_list, df):
  for i in range(0, len(factors_list)):
    if isinstance(factors_list[i], pd.DataFrame):
      print("found a dataframe")
      if factors_list[i].equals(df):
        print("equal to my dataframe, remove from list ")
        factors_list.pop(i)
  return factors_list    

def __eq__(node1, node2):
    return node1.name==node2.name and node1.parents==node2.parents and node1.children==node2.children

def object_in_list(object, list_of_objects):
  for i in range(0, len(list_of_objects)):
    if __eq__(object,list_of_objects[i]): 
      return True 

  return False


def toggle(var):
  if var == 0:
    return 1
  if var == 1:
    return 0

def boolean_truth_table(num_vars):
  truth_table = []
  num_vars = num_vars
  num_cols = num_vars
  var = 1
  num_rows = 2**num_vars
  for col_num in range(0,num_cols):
    truth_table.append([])
    col_num = col_num+1
    # print("col: {} toggle every {} values".format(col_num, 2**(num_vars-col_num)))
    for row_num in range(0, num_rows):
      if row_num%2**(num_vars-col_num)==0:
        var = toggle(var)
      truth_table[col_num-1].append(var)
      # print(truth_table[col_num-1])
      # print(var)
  return truth_table

def generate_truth_table(list_of_variables):
  d = dict()
  num_vars = len(list_of_variables)
  for i in range(0, num_vars): 
    d[str(list_of_variables[i])] = boolean_truth_table(len(list_of_variables))[i]
  df = pd.DataFrame(data=d)
  return df

def multiply_tabular_cpds(tabular_cpd1, tabular_cpd2, var_to_marginalize):
  df1 = tabular_cpd1.to_pandas_df()
  df1 = tabular_cpd1.to_pandas_df()

  vars_factor_1 = tabular_cpd1.get_factor().vars_in_factor()
  vars_factor_1.remove(var_to_marginalize)

  vars_factor_2 = tabular_cpd2.get_factor().vars_in_factor()
  vars_factor_2.remove(var_to_marginalize)

  total_vars = []
  total_vars.append(vars_factor_1[0])
  total_vars.append(vars_factor_2[0])
  output_table = generate_truth_table(total_vars) 

  vars_factor_1.append('Pr')
  vars_factor_2.append('Pr')

  result = pd.merge(df1, df2, on='B', how='inner')
  r0 = result[result['B']==0].Pr_x*result[result['B']==0].Pr_y
  r1 = (result[result['B']==1].Pr_x*result[result['B']==1].Pr_y).reset_index(drop=True)
  output_table['Pr']= r0.add(r1, fill_value=0)
  output_table
  return output_table

def multiply_tabular_cpds_v2(cpd_a, cpd_b):
  """
  input: two tabular cpds which can be in either TabularCPD or dataframe format
  output: dataframe which is the product of the two cpds multiplied by 
  their common term
  """
  if isinstance(cpd_a, TabularCPD):
    left = cpd_a.to_truth_table()
  else: 
    left = cpd_a
  if isinstance(cpd_b, TabularCPD):
    right = cpd_b.to_truth_table()
  else:
    right = cpd_b
  common_columns= np.intersect1d(left.columns, right.columns)
  common_columns= list(common_columns)
  common_columns.remove('Pr')
  result = pd.merge(left, right, on=common_columns, how='inner')
  result['Pr']=result['Pr_x'] *result['Pr_y']
  return result.drop(columns=['Pr_x', 'Pr_y'])

def df_marginalize(df, vars_to_marginalize):
  """
  input: a dataframe in truth table format with a "Pr" column - typically an output of `multiply_tabular_cpds_v2`
        , a list of variables to marginalize on 
  output: a dataframe with the removed column that you marginalized on
  """
  if isinstance(df, TabularCPD):
    df = df.to_truth_table()
    df_columns = list(df.columns)
    df_columns.remove('Pr')

    l1 = df_columns
    l2 = vars_to_marginalize
    l3 = [x for x in l1 if x not in l2]
    group_by_list = l3
    if len(group_by_list)>0:
      df = df.groupby(group_by_list).sum()
      return df.drop(columns=vars_to_marginalize).reset_index() 
  if isinstance(df, pd.DataFrame):
    df_columns = list(df.columns)
    df_columns.remove('Pr')
    l1 = df_columns
    l2 = vars_to_marginalize
    l3 = [x for x in l1 if x not in l2]
    group_by_list = l3
    if len(group_by_list)>0:
      df = df.groupby(group_by_list).sum()  
      return df.drop(columns=vars_to_marginalize).reset_index() 
  else: 
    return df 

def recursive_multiplication(list_of_cpds):
  product = None
  for i in range(0, len(list_of_cpds)):
    if i==0: 
      if isinstance(list_of_cpds[0], TabularCPD):
        product = list_of_cpds[0].to_truth_table()
      elif isinstance(list_of_cpds[0], pd.DataFrame):
        product = list_of_cpds[0]
    if i>0:
      product = multiply_tabular_cpds_v2(product, list_of_cpds[i])
  return product

In [0]:
class Node:
  def __init__(self, name, parents='', children=''):
    self.name = name
    self.parents = parents
    self.children = children

In [0]:
class MyBayesianModel: 
  def __init__(self, list_of_edges, nodes=[]):
    self.list_of_edges = list(list_of_edges)
    self.nodes = nodes # list(set([k for i in list_of_edges for k in i]))
    self.tabular_cpds=[]
    self.suggested_order=[]
    

  def get_suggested_order(self):
    if len(self.suggested_order)==0:
      variables = [''] *len(self.nodes)
      parents = np.zeros(len(self.nodes))
      children= np.zeros(len(self.nodes))

      for i in range(0, len(self.nodes)):
        variables[i]=self.nodes[i].name
        parents[i]=len(self.nodes[i].parents)
        children[i]=len(self.nodes[i].children)

      df = pd.DataFrame(variables, columns =['Variables']) 
      df['Parents'] = parents
      df['Children'] = children
      df['Total'] = df['Parents']+df['Children']
      suggested_order = list(df.sort_values(by=['Total'])['Variables'])
      self.suggested_order = suggested_order 
      return suggested_order
    else:
      return self.suggested_order
      
#1 construct a factor for each conditional probability
  def add_cpds(self, list_of_cpds):
      for i in range(0, len(list_of_cpds)):
        list_of_cpds[i].model = self
        # while the model doesn't have the full list of cpds, keep appending
        if (len(self.tabular_cpds) <= len(list_of_cpds)):
          self.tabular_cpds.append(list_of_cpds[i])
          edges_with_children=[item for item in list_of_cpds[i].model.list_of_edges if item[0] == list_of_cpds[i].variable]
    
        #adding nodes and parents to the model that it belongs to
          if len(edges_with_children)>0: 
            list_of_cpds[i].model.nodes.append(Node(list_of_cpds[i].variable, 
                              list_of_cpds[i].evidence, 
                              [item for item in list_of_cpds[i].model.list_of_edges if item[0] == list_of_cpds[i].variable][0][1]))
            
          elif len(edges_with_children)==0: 
            list_of_cpds[i].model.nodes.append(Node(list_of_cpds[i].variable, 
                              list_of_cpds[i].evidence ))
      self.dedupe_list_of_nodes()
  
  def dedupe_list_of_nodes(self):
    ##starts with an empty list and only adds items to it if they don't exist in the list
    empty_list=[]
    for i in range(0, len(self.nodes)):
      if not object_in_list(self.nodes[i], empty_list):
        empty_list.append(self.nodes[i])
    self.nodes = empty_list

  def print_all_factors(self):
    for i in range(0, len(self.tabular_cpds)):
      self.tabular_cpds[i].print_factor()

  def get_variables(self):
      list_of_tuples = list(self.list_of_edges)
      list_of_items = [item for t in list_of_tuples for item in t] 
      list_set = set(list_of_items) 
      # convert the set to the list 
      unique_list_of_vars = (list(list_set))
      print(unique_list_of_vars)

  def available_cpds(self):
    for i in range(0, len(self.tabular_cpds)):
      self.tabular_cpds[i].print_factor()

In [0]:
class Factor:
  def __init__(self, indep_var, dep_vars=[]):
    self.indep_var = indep_var
    self.dep_vars = dep_vars
  
  def print_factor(self):
    if len(self.dep_vars)>0:
      self.dep_vars = set(self.dep_vars)
      self.dep_vars = list(self.dep_vars)
      self.dep_vars.sort()

      dep_vars = str(self.dep_vars[0])
      for i in range (1, len(self.dep_vars)):
        dep_vars = dep_vars + "," + self.dep_vars[i]
      print("P({}|{})".format(self.indep_var, dep_vars))
    elif len(self.dep_vars)==0:
      print("P({})".format(self.indep_var))

  def vars_in_factor(self):
    factors = list()
    factors.append(str(self.indep_var))
    for i in range (0, len(self.dep_vars)):
      factors.append(self.dep_vars[i])
    
    list_set = set(factors) 
    # convert the set to the list 
    unique_list_of_vars = list(list_set)
    unique_list_of_vars.sort(reverse=False)

    return unique_list_of_vars

  def contains_var(self, variable):
    if variable in self.vars_in_factor():
      return True
    else:
      return False

  

In [0]:
class TabularCPD: 
  """
  input to initialize: 
        * variable - dependent variable
        * variable_card - how many possible values for dependent variable 
        * values - tabular probabilities 
        * evidence - independent variable 
        * evidence_card - list of possibilities for each of the dependent variables
  """
  def __init__(self, variable, variable_card, values, evidence='', evidence_card=''): 
    self.variable = variable 
    self.variable_card = variable_card
    self.values = values
    self.evidence = evidence
    self.evidence_card = evidence_card


  # Initializing factors of the CPD depending on the format (whether evidence is provided or not)
    if len(self.evidence)>0:
      self.factors = []
      self.factors.append(Factor(self.variable, self.evidence))
    
    if len(self.evidence)==0:
      self.factors = []
      self.factors.append(Factor(self.variable))
      
    self.all_variables = []
    self.all_variables.append(self.variable)
    for i in range (0, len(self.evidence)):
      self.all_variables.append(self.evidence[i])

    self.truth_table = generate_truth_table(self.all_variables)
    self.probabilities = []

    for i in range(0, len(self.values)):
      for j in range(0, len(self.values[i])):
        self.probabilities.append(self.values[i][j])

    probs = np.asarray(self.probabilities, dtype=np.float32)
    self.truth_table['Pr'] = probs

  def get_factor(self):
    """Returns all the factors associated with a TabularCPD"""
    return self.factors[0]

  def print_factor(self):
    self.factors[0].print_factor()
  
  def to_truth_table(self):
    self.truth_table = generate_truth_table(self.all_variables)

    self.probabilities = []

    for i in range(0, len(self.values)):
      for j in range(0, len(self.values[i])):
        self.probabilities.append(self.values[i][j])

    probs = np.asarray(self.probabilities, dtype=np.float32)
    self.truth_table['Pr'] = probs

    return self.truth_table

  def marginalize(self, var_to_marginalize): 
    #marginalize by "E"
    #var_to_marginalize = 'E'
    #keeping only the values to group by that are not the variable you want to marginalize on 
    group_by_list = list(filter(lambda a: a != var_to_marginalize, self.all_variables))
    df = self.truth_table.groupby(group_by_list).sum()
    df = df.drop(columns=[var_to_marginalize])
    return df 

  def to_pandas_df(self): 
    df = pd.DataFrame.from_records(self.values)
    if self.evidence!='':
      df = df.transpose()

    # for every column append the name of the tabular self variable 
    cols_in_df = len(df.columns)
    rows_in_df = len(df.index)

    for i in range(0, cols_in_df):
      df.rename(columns={ df.columns[i]: str(self.variable)+"_"+str(i) }, inplace = True)

    df.reset_index(inplace=True)

    if self.evidence!='':
      for i in range(0, rows_in_df):
        df.iloc[i,0] = str(self.evidence[0])+"_"+str(i)


    df.set_index('index')

    if self.evidence_card == [2,2]:
      df = pd.DataFrame.from_records(self.values)
      df = df.transpose()

      # for every column append the name of the tabular self variable 
      cols_in_df = len(df.columns)
      rows_in_df = len(df.index)

      for i in range(0, cols_in_df):
        df.rename(columns={ df.columns[i]: str(self.variable)+"_"+str(i) }, inplace = True)

      data = df
      df =generate_truth_table(self.evidence) 
      df['index']= str(self.evidence[0]) + "_" + df[str(self.evidence[0])].astype(str) + "_" + str(self.evidence[1]) + "_"+df[str(self.evidence[1])].astype(str)
      df = df[['index']]
      df_final = pd.concat([df, data], axis=1)
      df_final.set_index('index')
      return df_final.set_index('index')
    return df.set_index('index')

In [0]:
class MyVariableElimination:
  def __init__(self, model):
    self.model = model
    self.factors_list = self.model.tabular_cpds
    self.final_factor = []
    self.order = self.model.get_suggested_order()

  def query(self, desired_variables, evidence=dict()):
    self.desired_variables = desired_variables
    self.evidence = evidence

    print("Desired variables: {}".format(self.desired_variables))
    print("Evidence: {}".format(self.evidence))

    ##initializing order 
    if len(infer.evidence) == 0:
      self.order = self.model.get_suggested_order()
      for i in range(0, len(self.desired_variables)):
        try:
            self.order.remove(self.desired_variables[i])
        except:
          print("Please reinitialize your model")
      
      print("My initial list of factors is " + str(self.factors_list)+ " of size: {}".format(len(self.factors_list)))

            # for i in range(0, len(order)):
      for i in range(0, 5):
        print("Eliminating variable: {}".format(self.order[i]))
        cpds_to_multiply = self.get_cpds_to_multiply(self.factors_list, self.order[i])  
        product=recursive_multiplication(cpds_to_multiply)
        
        print("Marginalizing by:  {}".format(self.order[i]))
        resulting_factor = df_marginalize(product, self.order[i])
        print("Appending factor: \n {} of type {}".format(resulting_factor, type(resulting_factor)))
        print("Resulting factor is :{}")
        # return resulting_factor
        resulting_factor = df_to_cpd(resulting_factor)
        self.final_factor.append(resulting_factor)
        
        self.delete_cpds_from_list(cpds_to_multiply)
        if len(self.factors_list)==0:
          print("answer is ")
          resulting_factor
          break
        self.add_factor_to_list(resulting_factor)
      return resulting_factor.to_truth_table()
      # # for i in range(0, len(order)):
      # for i in range(0, 1):
      #   print("Eliminating variable: {}".format(self.order[i]))
      #   self.cpds_to_multiply = self.get_cpds_to_multiply(self.factors_list, self.order[i])  
      #   product=recursive_multiplication(self.cpds_to_multiply)
        
      #   print("Marginalizing by:  {}".format(self.order[i]))
      #   self.resulting_factor = df_marginalize(product, self.order[i])
      #   #print("Appending factor: \n {} of type {}".format(self.resulting_factor, type(self.resulting_factor)))
        
      #   print("Resulting factor is :{}".format(self.resulting_factor))
      #   # return resulting_factor
      #   self.resulting_factor = df_to_cpd(self.resulting_factor)
      #   self.final_factor.append(self.resulting_factor)
        
      #   self.delete_cpds_from_list(self.cpds_to_multiply)
      #   if len(self.factors_list)==0:
      #     print("answer is ")
      #     df = self.resulting_factor.to_truth_table().copy()
      #     if self.resulting_factor.to_truth_table()['Pr'].sum() != 1:
      #       print("Normalizing...")
            
      #       df['Pr']= df['Pr'].div(df['Pr'].sum())
      #     print(df = self.resulting_factor.to_truth_table().cop)
      #     break
      #   self.add_factor_to_list(self.resulting_factor)
      # return self.resulting_factor.to_truth_table()

    if len(infer.evidence) > 0:
      print("Conditonal query wanted")
      print("Choose an elimination ordering ")
      self.order = self.model.get_suggested_order()
      print("The order for elimination is: {}".format(self.order))

      print("\nReplacing evidence with restriction")
      self.replace_evidence_with_restriction()
      self.factors_list = self.model.tabular_cpds

      self.choose_order()
      print("New order is: {}, which is a list of length {}".\
            format(self.order, len(self.order)))

      for i in range(0, len(self.order)):
        print("This is the {}th iteration...".format(i))
        print("Eliminating {}".format(self.order[i]))

        self.cpds_to_multiply = self.get_cpds_to_multiply(self.factors_list, self.order[i])  

        print("The cpds to multiply are: ")
        print(type(self.cpds_to_multiply))
        for k in range(0, len(self.cpds_to_multiply)):
          print(self.cpds_to_multiply[k].to_truth_table())

        self.product=recursive_multiplication(self.cpds_to_multiply)
        print("The product of recursive multiplication is \n {} of type {}".format(self.product, type(self.product)))

        print("Marginalizing by {}".format(self.order[i]))
        self.resulting_factor = df_marginalize(self.product, self.order[i])

        print("The resulting factor is \n{} of type {}".format(self.resulting_factor, type(self.resulting_factor)))
        print("The factors in my list are:\n")
        for j in range(0, len(self.factors_list)):
          print(self.factors_list[j].to_truth_table())
 
        print("Resulting factor is :\n {}".format(self.resulting_factor))
        self.resulting_factor = df_to_cpd(self.resulting_factor)

        print("Resulting factor after converting to cpd is :\n {}".format(self.resulting_factor.to_truth_table()))


        if self.resulting_factor.to_truth_table()['Pr'].sum() == self.resulting_factor.to_truth_table()['Pr'].count(): 
          print("This resulting factor is irrelevant, so won't get appended to self.factors_list")
        

        self.delete_cpds_from_list(self.cpds_to_multiply)

        df = self.resulting_factor.to_truth_table().copy()

        if len(self.factors_list)==0:
          print("answer is ")
          df = self.resulting_factor.to_truth_table().copy()
          if self.resulting_factor.to_truth_table()['Pr'].sum() != 1:
            print("Normalizing...")
            df['Pr']= df['Pr'].div(df['Pr'].sum())
            print(df)
            return df
          break

        if self.resulting_factor.to_truth_table()['Pr'].sum() != self.resulting_factor.to_truth_table()['Pr'].count(): 
          print("Factor is not irrelevant.\n")
          self.add_factor_to_list(self.resulting_factor)
          #return self.resulting_factor.to_truth_table()
        
        self.remove_irrelevant_factors()

      #Factors left that have to do with the variable that we want 
      self.product =recursive_multiplication(self.factors_list)
      self.resulting_factor = df_to_cpd(self.product)
      print("answer is ")
      df = self.resulting_factor.to_truth_table().copy()
      if self.resulting_factor.to_truth_table()['Pr'].sum() != 1:
        print("Normalizing...")
        df['Pr']= df['Pr'].div(df['Pr'].sum())
        print(df)
        return df


    return df

  def remove_irrelevant_factors(self):
    print("self.factors_list before removing irrelevant factors is ")
    for i in range(0, len(self.factors_list)):
      print(self.factors_list[i].to_truth_table())

    indices_to_pop = []
    for i in range(0, len(self.factors_list)):
      if self.factors_list[i].to_truth_table()['Pr'].sum() == self.factors_list[i].to_truth_table()['Pr'].count():
        # print("adding {}".format(i))
        indices_to_pop.append(i)

    indices_to_pop
    for j in range(0, len(indices_to_pop)):
      self.factors_list.pop(indices_to_pop[j])

    return self.factors_list

  def choose_order(self):
    print("desired variables")
    for i in range(0, len(self.desired_variables)):
      print(self.desired_variables[i])
      self.order.remove(self.desired_variables[i])
    for key, value in self.evidence.items() :
      print (key, value)
      self.order.remove(key)
      

  def get_cpds_to_multiply(self, factors_list, variable_to_eliminate):
    current_variable = variable_to_eliminate
    cpds_to_multiply = []
    for i in range(0, len(self.model.tabular_cpds)):
      if isinstance(self.model.tabular_cpds[i], TabularCPD):
        if self.model.tabular_cpds[i].get_factor().contains_var(current_variable):
          cpds_to_multiply.append(self.model.tabular_cpds[i])
    ##see if dataframe contains a variable and add it to the cpds_to_multiply list
      elif not isinstance(self.model.tabular_cpds[i], TabularCPD):
        df_vars = list(self.model.tabular_cpds[i].columns)
        if current_variable in df_vars: 
          cpds_to_multiply.append(self.model.tabular_cpds[i])
    print("Multiplying:...")
    for i in range (0, len(cpds_to_multiply)):
      if isinstance(cpds_to_multiply[i], TabularCPD):
        cpds_to_multiply[i].print_factor()
      if isinstance(cpds_to_multiply[i], pd.DataFrame):
        print(cpds_to_multiply[i])

    return cpds_to_multiply

  def delete_cpds_from_list(self, cpds_to_multiply):
    for i in range(0, len(cpds_to_multiply)):
      print("List size of cpds_to_multiply is {}".format(len(cpds_to_multiply)))
    # if it's a cpd, remove it from the list 
      if isinstance(cpds_to_multiply[i], TabularCPD):
        print("Removing from self.factors_list:")
        cpds_to_multiply[i].print_factor()
        self.factors_list.remove(cpds_to_multiply[i])
      #if it's a data frame, find the equivalent one in the factors list and pop it   
      if isinstance(cpds_to_multiply[i], pd.DataFrame):
        self.factors_list = delete_df_from_list(self.factors_list, cpds_to_multiply[i])

  def add_factor_to_list(self, resulting_factor):
    print("List size of self.factors_list is {}".format(len(self.factors_list)))
    print("There are {} Factors left in self.factors_list ".format(len(self.factors_list)))
    for i in range(0, len(self.factors_list)):
      # print(self.factors_list[i])
      print(self.factors_list[i].to_truth_table())

    print("Adding new factor to list")
    print(resulting_factor.to_truth_table())
    self.factors_list.append(resulting_factor)
    self.final_factor.append(resulting_factor)

    print("There are {} Factors left in self.factors_list".format(len(self.factors_list)))
    for i in range(0, len(self.factors_list)):
      # print(self.factors_list[i])
      print(self.factors_list[i].to_truth_table())

  def replace_evidence_with_restriction(self):
      for i in range(0, len(list(self.evidence.keys()))):
        evidence = list(self.evidence.keys())[i]
        print("Evidence is {}".format(evidence))
        evidence_value = list(self.evidence.values())[i]
        print("Evidence value is {} ".format(evidence_value))
        for i in range(0, len(self.model.tabular_cpds)):
          print(self.model.tabular_cpds[i].get_factor().contains_var(evidence))
          self.model.tabular_cpds[i].print_factor()
          if self.model.tabular_cpds[i].get_factor().contains_var(evidence):
            df = self.model.tabular_cpds[i].to_truth_table()
            print("New factor after replacing evidence {} with value  {} is:".format(evidence, evidence_value))
            print(df[df[evidence]==evidence_value])
            print("cpd representation of that is: \n")
            print(df_to_cpd(df[df[evidence]==evidence_value]).to_truth_table())
            self.new_cpd = df_to_cpd(df[df[evidence]==evidence_value])
            self.new_df = df[df[evidence]==evidence_value]
            self.model.tabular_cpds[i] = df_to_cpd(df[df[evidence]==evidence_value])


### Test data

In [0]:
model = MyBayesianModel([('S1', 'O1'), ('S1', 'S2'), ('S2', 'O2'), ('S2', 'S3'), ('S3', 'O3')])

### Query 1 

In [9]:
cpd_s1 = TabularCPD(variable='S1', variable_card=2, values=[[1,0]])
cpd_s2 = TabularCPD(variable='S2', variable_card=2,
                   values=[[0.8, 0],
                           [0.2, 1]],
                   evidence=['S1'],
                   evidence_card=[2])
cpd_s3 = TabularCPD(variable='S3', variable_card=2,
                   values=[[0.8, 0],
                           [0.2, 1]],
                   evidence=['S2'],
                   evidence_card=[2])

cpd_o1 = TabularCPD(variable='O1', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S1'],
                   evidence_card=[2])

cpd_o2 = TabularCPD(variable='O2', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S2'],
                   evidence_card=[2])


cpd_o3 = TabularCPD(variable='O3', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S3'],
                   evidence_card=[2])

model.add_cpds([cpd_s1, cpd_s2, cpd_s3, cpd_o1, cpd_o2, cpd_o3])

infer = MyVariableElimination(model)

S3_dist = infer.query(['S3'])
S3_dist

Desired variables: ['S3']
Evidence: {}
My initial list of factors is [<__main__.TabularCPD object at 0x7f917deedf98>, <__main__.TabularCPD object at 0x7f917deedf60>, <__main__.TabularCPD object at 0x7f919548b748>, <__main__.TabularCPD object at 0x7f917dec8208>, <__main__.TabularCPD object at 0x7f917dec8da0>, <__main__.TabularCPD object at 0x7f917dec8898>] of size: 6
Eliminating variable: O1
Multiplying:...
P(O1|S1)
Marginalizing by:  O1
Appending factor: 
    S1   Pr
0   0  1.0
1   1  1.0 of type <class 'pandas.core.frame.DataFrame'>
Resulting factor is :{}
0
S1
0    0
1    1
Name: S1, dtype: int64
List size of cpds_to_multiply is 1
Removing from self.factors_list:
P(O1|S1)
List size of self.factors_list is 5
There are 5 Factors left in self.factors_list 
   S1   Pr
0   0  1.0
1   1  0.0
   S2  S1   Pr
0   0   0  0.8
1   0   1  0.0
2   1   0  0.2
3   1   1  1.0
   S3  S2   Pr
0   0   0  0.8
1   0   1  0.0
2   1   0  0.2
3   1   1  1.0
   O2  S2   Pr
0   0   0  0.5
1   0   1  0.0
2   1 

,S3,Pr
0,0,0.64
1,1,0.36


### Query 2

In [10]:
model = MyBayesianModel([('S1', 'O1'), ('S1', 'S2'), ('S2', 'O2'), ('S2', 'S3'), ('S3', 'O3')])
cpd_s1 = TabularCPD(variable='S1', variable_card=2, values=[[1,0]])
cpd_s2 = TabularCPD(variable='S2', variable_card=2,
                   values=[[0.8, 0],
                           [0.2, 1]],
                   evidence=['S1'],
                   evidence_card=[2])
cpd_s3 = TabularCPD(variable='S3', variable_card=2,
                   values=[[0.8, 0],
                           [0.2, 1]],
                   evidence=['S2'],
                   evidence_card=[2])

cpd_o1 = TabularCPD(variable='O1', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S1'],
                   evidence_card=[2])

cpd_o2 = TabularCPD(variable='O2', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S2'],
                   evidence_card=[2])


cpd_o3 = TabularCPD(variable='O3', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S3'],
                   evidence_card=[2])

model.add_cpds([cpd_s1, cpd_s2, cpd_s3, cpd_o1, cpd_o2, cpd_o3])

infer = MyVariableElimination(model)
df = infer.query(['S3'], {'O2':1})
df 


Desired variables: ['S3']
Evidence: {'O2': 1}
Conditonal query wanted
Choose an elimination ordering 
The order for elimination is: ['O1', 'O2', 'O3', 'S1', 'S2', 'S3']

Replacing evidence with restriction
Evidence is O2
Evidence value is 1 
False
P(S1)
False
P(S2|S1)
False
P(S3|S2)
False
P(O1|S1)
True
P(O2|S2)
New factor after replacing evidence O2 with value  1 is:
   O2  S2   Pr
2   1   0  0.5
3   1   1  1.0
cpd representation of that is: 

There's a column of 1s for variable O2
Removing O2
0
O2
2    1
3    1
Name: O2, dtype: int64
1
S2
2    0
3    1
Name: S2, dtype: int64
   S2   Pr
0   0  0.5
1   1  1.0
There's a column of 1s for variable O2
Removing O2
0
O2
2    1
3    1
Name: O2, dtype: int64
1
S2
2    0
3    1
Name: S2, dtype: int64
There's a column of 1s for variable O2
Removing O2
0
O2
2    1
3    1
Name: O2, dtype: int64
1
S2
2    0
3    1
Name: S2, dtype: int64
False
P(O3|S3)
desired variables
S3
O2 1
New order is: ['O1', 'O3', 'S1', 'S2'], which is a list of length 4
This 

,S3,Pr
0,0,0.533333
1,1,0.466667


### Query 3

In [11]:
model = MyBayesianModel([('S1', 'O1'), ('S1', 'S2'), ('S2', 'O2'), ('S2', 'S3'), ('S3', 'O3')])

cpd_s1 = TabularCPD(variable='S1', variable_card=2, values=[[1,0]])
cpd_s2 = TabularCPD(variable='S2', variable_card=2,
                   values=[[0.8, 0],
                           [0.2, 1]],
                   evidence=['S1'],
                   evidence_card=[2])
cpd_s3 = TabularCPD(variable='S3', variable_card=2,
                   values=[[0.8, 0],
                           [0.2, 1]],
                   evidence=['S2'],
                   evidence_card=[2])

cpd_o1 = TabularCPD(variable='O1', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S1'],
                   evidence_card=[2])

cpd_o2 = TabularCPD(variable='O2', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S2'],
                   evidence_card=[2])


cpd_o3 = TabularCPD(variable='O3', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S3'],
                   evidence_card=[2])

model.add_cpds([cpd_s1, cpd_s2, cpd_s3, cpd_o1, cpd_o2, cpd_o3])

infer = MyVariableElimination(model)
# df = infer.query(['S3'], {'O2':1})
# df 

# S3_dist = infer.query(['S3'])
# S3_dist

df = infer.query(['S3'], evidence={'O1':1, 'O2':0, 'O3':1})
df 

Desired variables: ['S3']
Evidence: {'O1': 1, 'O2': 0, 'O3': 1}
Conditonal query wanted
Choose an elimination ordering 
The order for elimination is: ['O1', 'O2', 'O3', 'S1', 'S2', 'S3']

Replacing evidence with restriction
Evidence is O1
Evidence value is 1 
False
P(S1)
False
P(S2|S1)
False
P(S3|S2)
True
P(O1|S1)
New factor after replacing evidence O1 with value  1 is:
   O1  S1   Pr
2   1   0  0.5
3   1   1  1.0
cpd representation of that is: 

There's a column of 1s for variable O1
Removing O1
0
O1
2    1
3    1
Name: O1, dtype: int64
1
S1
2    0
3    1
Name: S1, dtype: int64
   S1   Pr
0   0  0.5
1   1  1.0
There's a column of 1s for variable O1
Removing O1
0
O1
2    1
3    1
Name: O1, dtype: int64
1
S1
2    0
3    1
Name: S1, dtype: int64
There's a column of 1s for variable O1
Removing O1
0
O1
2    1
3    1
Name: O1, dtype: int64
1
S1
2    0
3    1
Name: S1, dtype: int64
False
P(O2|S2)
False
P(O3|S3)
Evidence is O2
Evidence value is 0 
False
P(S1)
False
P(S2|S1)
False
P(S3|S2)
Fal

,S3,Pr
0,0,0.666667
1,1,0.333333


In [14]:
! pip install pgmpy
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = BayesianModel([('S1', 'O1'), ('S1', 'S2'), ('S2', 'O2'), ('S2', 'S3'), ('S3', 'O3')])

cpd_s1 = TabularCPD(variable='S1', variable_card=2, values=[[1,0]])
cpd_s2 = TabularCPD(variable='S2', variable_card=2,
                   values=[[0.8, 0],
                           [0.2, 1]],
                   evidence=['S1'],
                   evidence_card=[2])
cpd_s3 = TabularCPD(variable='S3', variable_card=2,
                   values=[[0.8, 0],
                           [0.2, 1]],
                   evidence=['S2'],
                   evidence_card=[2])

cpd_o1 = TabularCPD(variable='O1', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S1'],
                   evidence_card=[2])

cpd_o2 = TabularCPD(variable='O2', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S2'],
                   evidence_card=[2])


cpd_o3 = TabularCPD(variable='O3', variable_card=2,
                   values=[[0.5, 0],
                           [0.5, 1]],
                   evidence=['S3'],
                   evidence_card=[2])

model.add_cpds(cpd_s1, cpd_s2, cpd_s3, cpd_o1, cpd_o2, cpd_o3)
infer2 = VariableElimination(model)


S3_dist = infer2.query(['S3'])
S3_dist



Eliminating: O1: 100%|██████████| 5/5 [00:00<00:00, 315.30it/s]


<DiscreteFactor representing phi(S3:2) at 0x7f9124f99898>

### Query 1 with PGMPY

In [16]:
print(S3_dist)

+-------+-----------+
| S3    |   phi(S3) |
+=======+===========+
| S3(0) |    0.6400 |
+-------+-----------+
| S3(1) |    0.3600 |
+-------+-----------+


### Query 2 with PGMPY

In [17]:
df = infer2.query(['S3'], {'O2':1})
print(df)


Eliminating: S2: 100%|██████████| 4/4 [00:00<00:00, 404.35it/s]

+-------+-----------+
| S3    |   phi(S3) |
+=======+===========+
| S3(0) |    0.5333 |
+-------+-----------+
| S3(1) |    0.4667 |
+-------+-----------+


### Query 3

In [18]:
print(infer2.query(['S3'], evidence={'O1':1, 'O2':0, 'O3':1}))

Eliminating: S2: 100%|██████████| 2/2 [00:00<00:00, 299.60it/s]

+-------+-----------+
| S3    |   phi(S3) |
+=======+===========+
| S3(0) |    0.6667 |
+-------+-----------+
| S3(1) |    0.3333 |
+-------+-----------+
